In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import urllib.request
import re
import os

In [ ]:
html_text = requests.get('https://batdongsan.vn/ban-nha/').text
soup = BeautifulSoup(html_text, "lxml")

In [ ]:
pages_url=[]

In [ ]:
def get_dom(url):
    with urllib.request.urlopen(url) as response:
        html = response.read()
        html = html.decode('utf-8')
    response.close()
    soup = BeautifulSoup(html, 'lxml')
    return soup

In [ ]:
number = soup.find('ul', class_ = 'uk-pagination')
page_number = int(number.find_all('a')[-1]['data-ci-pagination-page'])
page_number

In [ ]:
for i in range(420, page_number+1): # link của n trang
  page_url = "https://batdongsan.vn/ban-nha/p" + str(i)
  pages_url.append(page_url)

In [ ]:
def get_ID(dom):
    try:
        temp = dom.find('div', class_='project-global-object-block-003 information-custom')
        id_element = temp.find('strong', string = 'Mã tin:')
        id = id_element.next_sibling.strip().split(' ')[0]
    except Exception as e: 
        id = "ID is not available"
    return id 

In [ ]:
def get_Price(dom):
    try:
        temp = dom.find('div', class_='project-global-object-block-003 information-custom')
        price = temp.find('strong', class_ = "price").text.strip()
    except Exception as e: 
        price = "Price is not available"
    return price

In [ ]:
def get_Location(dom):
    try:
        temp = dom.find('ul', class_ = "uk-breadcrumb")
        temp1 = temp.find_all('li')
        result = temp1[3].text + ', ' + temp1[2].text
    except Exception as e: 
        result = "Location is not available"
    return result

In [ ]:
def get_Area(dom):
    try:
        temp = dom.find('div', class_='project-global-object-block-003 information-custom')
        area_element = temp.find('strong', string = 'Diện tích:')
        area = area_element.next_sibling.strip().split(' ')[0]
    except Exception as e: 
        area = "Area is not available"
    return area

In [ ]:
def get_Bathroom(dom):
    try:
        temp = dom.find('div', class_='project-global-object-block-003 information-custom')
        bathroom_element = temp.find('strong', string = 'Phòng WC:')
        bathroom = bathroom_element.next_sibling.strip().split(' ')[0]
    except Exception as e: 
        bathroom = "Bathroom is not available"
    return bathroom

In [ ]:
def get_Bedroom(dom):
    try:
        temp = dom.find('div', class_='project-global-object-block-003 information-custom')
        bedroom_element = temp.find('strong', string = 'Phòng ngủ:')
        bedroom = bedroom_element.next_sibling.strip().split(' ')[0]
    except Exception as e:
        bedroom = "Bedroom is not available"
    return bedroom

In [ ]:
def get_Content(dom):
    try:
        a = dom.find('div', class_='project-global-object-block-003 information-custom')
        content1 = a.span.text
        temp = dom.find('div',class_ = 'project-global-object-block-003 block-custom').find('div', class_ ='content')
        content2 = temp.text.replace('\n','').replace('\r','').replace('+','').strip()
        content = content1 + ' ' + content2
        
    except Exception as e:
        content = "NULL"
    return content

In [ ]:
def get_Picture(dom,ID):
    try:
      wrapper = dom.find("div",class_='uk-slider uk-grid uk-grid-small uk-grid-width-1-3 uk-grid-width-medium-1-4')
      images = wrapper.find_all("div",class_='image cover')
      if not os.path.exists('./download/'):
        os.makedirs('./download/')
      for image in images:
        imgData = image.find('img')['src']
        if("data:image" not in imgData):
          if(imgData):
            downloadPath = './download/'
            filename = imgData.split('/')[-1]
            response = requests.get(imgData)

            file = open(downloadPath + ID + "-" + filename, "wb")
            file.write(response.content)
            file.close()
    except Exception as e:
      print("An exception occurred:", e)

In [ ]:
def regex_content(content):
    mat_tien = 0
    so_tang = 'NULL'
    so_pn = 'NULL'
    so_wc = 'NULL'
    dien_tich = "Area is not available"

    if content != 'NULL': 
        a = re.search(r"mặt tiền|mặt phố|mặt đường",content,re.IGNORECASE)
        if a:
            mat_tien = 1
                    
        tang=re.findall(r"tầng \d{1,2}|\d{1,2} tầng",content,re.IGNORECASE)
        tret_lau=re.findall(r"\d{1,2} lầu|\d{1,2} lửng",content,re.IGNORECASE)
        b1=re.findall("\d+"," ".join(tang))
        b2=re.findall("\d+"," ".join(tret_lau))
        if len(b1)!=0:
            b1=list(map(int,b1))
            so_tang = max(b1)
        elif len(b2)!=0:
            b2=list(map(int,b2))
            so_tang = max(b2)+1  #lầu thì luôn có trệt => +1
            
        c = re.findall(r"(\d+)\s*(?=phòng ngủ|pn|ngủ)",content,re.IGNORECASE)
        if len(c)!=0:
            c=list(map(int,c))
            so_pn = max(c)
        
        d = re.findall(r"(\d+)\s*(?=phòng vệ sinh|wc|nhà vệ sinh|nvs|pvs|vệ sinh|vs)",content,re.IGNORECASE)
        if len(d)!=0:
            d=list(map(int,d))
            so_wc = max(d)

        e = re.search(r"\d{1,3}\s*(?=m2|m²)|\d{1,3}\s*x\s*\d{1,3}",content,re.IGNORECASE)
        if e:
            dien_tich=1
            for i in list(map(int,re.findall(r"\d+",e.group()))):
                dien_tich*=i

    return  mat_tien, so_tang, so_pn, so_wc, dien_tich
    

In [ ]:
data_csv = []
for link_page in pages_url:
  soup_temp = get_dom(link_page)  # get_dom page trang
  urls = soup_temp.find('div', class_ = 'uk-grid uk-grid-small uk-grid-width-1-1')
  actual_link = urls.find_all('div', class_ = 'name')

  for data in actual_link:
    data_url = data.a['href']
    link_dom = get_dom(data_url)
    ID = get_ID(link_dom)
    if ID != 'ID is not available':
        get_Picture(link_dom,ID)
    content = get_Content(link_dom)
    price = get_Price(link_dom)
    area = get_Area(link_dom)
    location = get_Location(link_dom)
    bedroom = get_Bedroom(link_dom)
    bathroom = get_Bathroom(link_dom)
    mat_tien, so_tang, so_pn, so_wc, dien_tich = regex_content(content)
    if bedroom == 'Bedroom is not available' :
        bedroom = so_pn
    if bathroom == 'Bathroom is not available' :
        bathroom = so_wc
    if area == 'Area is not available' :
        area = dien_tich
    
    data_csv.append([ID, location, area, bedroom, bathroom, mat_tien, so_tang, price])

df = pd.DataFrame(data_csv, columns=['ID', 'Location', 'Area', 'Bedroom', 'Bathroom','Frontage', 'Floors', 'Price'])
df.to_csv('data.csv')
    